In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_df = pd.read_csv("train_processed.csv")
test_df = pd.read_csv("test_processed.csv")

X = train_df.drop(columns=['id', 'y', 'shares'])
y = train_df['y']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [31]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from xgboost import XGBClassifier

# 최적 하이퍼파라미터로 모델 정의
best_params = {
    'n_estimators': 487,
    'max_depth': 3,
    'learning_rate': 0.022783546678795837,
    'subsample': 0.9865934283784953,
    'colsample_bytree': 0.6270846419992482,
    'gamma': 0.8172927176529761,
    'min_child_weight': 4,
    'eval_metric': 'auc',
    'random_state': 42,
    'tree_method': 'hist',
    # CPU 사용을 원하면 'hist'
    # GPU를 사용하려면 'device': 'cuda' 추가
    'device': 'cuda'
}

# 최종 모델 학습
final_model = XGBClassifier(**best_params)
final_model.fit(X_train, y_train)

# 검증 데이터에 대해 예측 수행
y_pred_valid = final_model.predict(X_valid)
y_prob_valid = final_model.predict_proba(X_valid)[:, 1]  # 클래스 1일 확률

# 성능 평가 (Accuracy, F1 Score, AUC)
accuracy = accuracy_score(y_valid, y_pred_valid)
f1 = f1_score(y_valid, y_pred_valid)
auc = roc_auc_score(y_valid, y_prob_valid)

# 세 가지 지표의 평균 계산
mean_metric = (accuracy + f1 + auc) / 3

# 성능 결과 출력
print("Final Tuned Model Accuracy:", accuracy)
print("Final Tuned Model F1 Score:", f1)
print("Final Tuned Model AUC:", auc)
print("Mean Evaluation Metric (Accuracy + F1 + AUC) / 3:", mean_metric)


Final Tuned Model Accuracy: 0.6765765765765765
Final Tuned Model F1 Score: 0.6740807989105765
Final Tuned Model AUC: 0.7298996619142015
Mean Evaluation Metric (Accuracy + F1 + AUC) / 3: 0.6935190124671182


In [32]:
# 테스트 데이터에 대해 예측 수행
X_test = test_df.drop(columns=['id'])  # 'id' 컬럼을 제외한 테스트 데이터
y_pred_test = final_model.predict(X_test)
y_prob_test = final_model.predict_proba(X_test)[:, 1]

# 예측 결과를 test_df에 추가
test_df['y_predict'] = y_pred_test
test_df['y_prob'] = y_prob_test

# 예측 결과를 CSV 파일로 저장
output_path = '/content/prediction.csv'
test_df.to_csv(output_path, index=False)

# 예측된 결과 출력 (파일에 저장된 예시)
print(test_df[['id', 'y_predict', 'y_prob']].head())

      id  y_predict    y_prob
0   4979          0  0.272374
1  15552          0  0.430670
2  29370          1  0.576737
3  37272          0  0.273311
4   6836          1  0.524885


In [33]:
from google.colab import files
files.download(output_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>